## Exercise 01. Basic operations ##

## Imports ##

In [16]:
import pandas as pd

## Task ##

## 1. Create DataFrame ##
1. Создайте датафрейм с именем views с двумя столбцами: datetime и user, считав данные из feed-views.log.
   + Преобразуйте datetime в тип данных datetime64[ns].
   + Извлеките год, месяц, день, час, минуту и секунду из значений столбца datetime в новые столбцы.

In [17]:
views = pd.read_csv("../data/feed-views.log", sep='\t', header=None, names=['datetime', 'user'], engine='python')
print('Before: ', views['datetime'].dtype)
views['datetime'] = pd.to_datetime(views['datetime'])
print('After: ', views['datetime'].dtype)

Before:  object
After:  datetime64[ns]


In [18]:
views.head()

new_cols = ["year", "month", "day", "hour", "minute", "second"]
views[new_cols] = views['datetime'].apply(lambda data_obj: [getattr(data_obj, attr) for attr in new_cols]).apply(pd.Series)

views.head()

,datetime,user,year,month,day,hour,minute,second
0,2020-04-17 12:01:08.463179,artem,2020,4,17,12,1,8
1,2020-04-17 12:01:23.743946,artem,2020,4,17,12,1,23
2,2020-04-17 12:27:30.646665,artem,2020,4,17,12,27,30
3,2020-04-17 12:35:44.884757,artem,2020,4,17,12,35,44
4,2020-04-17 12:35:52.735016,artem,2020,4,17,12,35,52


## 2. New column ##

2. Создайте новый столбец daytime.
   + Присвойте определённое значение времени суток в зависимости от интервала часа. Например, присвойте "afternoon", если час больше 11 и меньше или равен 17.
   + 0–3:59 = ночь, 4–6:59 = раннее утро, 7–10:59 = утро, 11–16:59 = дневное время, 17–19:59 = ранний вечер, 20–23:59 = вечер.
   + Используйте метод cut для решения этой задачи.
   + Назначьте столбец user в качестве индекса.

In [19]:
views['daytime'] = pd.cut(views['hour'], bins=[0, 4, 7, 11, 17, 20, 24], right=False, labels=['night', 'early morning', 'morning', 'afternoon', 'early evening', 'evening'])
views.tail()

,datetime,user,year,month,day,hour,minute,second,daytime
1071,2020-05-21 18:45:20.441142,valentina,2020,5,21,18,45,20,early evening
1072,2020-05-21 23:03:06.457819,maxim,2020,5,21,23,3,6,evening
1073,2020-05-21 23:23:49.995349,pavel,2020,5,21,23,23,49,evening
1074,2020-05-21 23:49:22.386789,artem,2020,5,21,23,49,22,evening
1075,2020-05-22 10:36:14.662600,artem,2020,5,22,10,36,14,morning


In [20]:
views.set_index('user', inplace=True)
print('New index_col: ', views.index.name)

New index_col:  user


## 3. Count elements ##

3. Подсчитайте количество элементов в вашем датафрейме.
   + Используйте метод count().
   + Подсчитайте количество элементов в каждой категории времени суток с помощью метода value_counts().

In [21]:
views.count()

datetime    1076
year        1076
month       1076
day         1076
hour        1076
minute      1076
second      1076
daytime     1076
dtype: int64

In [22]:
views['daytime'].value_counts()

daytime
evening          509
afternoon        252
early evening    145
night            129
morning           36
early morning      5
Name: count, dtype: int64

## 4. Sort by time ##

4. Отсортируйте значения в датафрейме по часам, минутам и секундам одновременно в порядке возрастания, а не по отдельности.

In [23]:
views.sort_values(by=['hour', 'minute', 'second'], inplace=True)
views.head()

,datetime,year,month,day,hour,minute,second,daytime
user,,,,,,,,
valentina,2020-05-15 00:00:13.222265,2020,5,15,0,0,13,night
valentina,2020-05-15 00:01:05.153738,2020,5,15,0,1,5,night
pavel,2020-05-12 00:01:27.764025,2020,5,12,0,1,27,night
pavel,2020-05-12 00:01:38.444917,2020,5,12,0,1,38,night
pavel,2020-05-12 00:01:55.395042,2020,5,12,0,1,55,night


## 5. Max, Min, Mode ##

5. Вычислите минимум и максимум для часов и моду для категорий времени суток.
   + Вычислите максимальный час для строк, где время суток — ночь.
   + Вычислите минимальный час для строк, где время суток — утро.
   + Кроме того, выясните, кто посещал страницу в эти часы, и приведите один пример.
   + Вычислите моду для часа и времени суток.

In [24]:
max_night_hour = pd.Series(views['hour'][views['daytime'] == 'night']).max()
min_morning_hour = pd.Series(views['hour'][views['daytime'] == 'morning']).min()
print(f'Results:\n\
Min for hours: {views['hour'].min()}\n\
Max for hours: {views['hour'].max()}\n\
Mode for daytime: {views['daytime'].mode().values[0]}\n\
Max night hour: {max_night_hour}\n\
Min morning hour: {min_morning_hour}\n\
Max night hour visitor: {pd.Series(views.index.to_series()[views['hour'] == max_night_hour]).head(1).values[0]}\n\
Min morning hour visitor: {pd.Series(views.index.to_series()[views['hour'] == min_morning_hour]).head(1).values[0]}\n\
Mode for hour: {views['hour'].mode().values[0]}\n\
Mode for daytime: {views['daytime'].mode().values[0]}')

Results:
Min for hours: 0
Max for hours: 23
Mode for daytime: evening
Max night hour: 3
Min morning hour: 8
Max night hour visitor: konstantin
Min morning hour visitor: alexander
Mode for hour: 22
Mode for daytime: evening


## 6. 3 earliest and 3 latest ##

6. Покажите три самых ранних и три самых поздних часа дня с соответствующими именами пользователей, используя nsmallest() и nlargest().

In [25]:
views['hour'].nsmallest(3, keep='first')

user
valentina    0
valentina    0
pavel        0
Name: hour, dtype: int64

In [26]:
views['hour'].nlargest(3, keep='last')

user
alexander    23
valentina    23
pavel        23
Name: hour, dtype: int64

Или выбрать только уникальные часы:

In [27]:
views['hour'].drop_duplicates(keep='first').nsmallest(3, keep='first')

user
valentina     0
pavel         1
konstantin    2
Name: hour, dtype: int64

In [28]:
views['hour'].drop_duplicates(keep='last').nlargest(3, keep='last')

user
alexander    23
ekaterina    22
anatoliy     21
Name: hour, dtype: int64

## 7. Base statistics ##

7. Используйте метод describe() для получения базовой статистики по столбцам.
   + Чтобы найти самый популярный интервал посещений, вычислите интерквартильный размах для часа, извлекая значения из результата метода describe() и сохраняя их в переменную iqr.

In [29]:
views.describe()

,datetime,year,month,day,hour,minute,second
count,1076,1076.0,1076.000000,1076.000000,1076.000000,1076.000000,1076.000000
mean,2020-05-10 09:00:41.211420672,2020.0,4.870818,13.552974,16.249071,29.629182,29.500929
min,2020-04-17 12:01:08.463179,2020.0,4.000000,1.000000,0.000000,0.000000,0.000000
25%,2020-05-10 01:13:49.857472,2020.0,5.000000,11.000000,13.000000,14.000000,14.000000
50%,2020-05-11 22:48:35.302552832,2020.0,5.000000,13.000000,19.000000,29.000000,30.000000
75%,2020-05-14 14:44:34.749530624,2020.0,5.000000,15.000000,22.000000,46.000000,45.000000
max,2020-05-22 10:36:14.662600,2020.0,5.000000,30.000000,23.000000,59.000000,59.000000
std,NaN,0.0,0.335557,4.906567,6.955490,17.689388,17.405506


In [30]:
stats = views['hour'].describe()
iqr = stats['75%'] - stats['25%']
print('Interquartile range (IQR): ', iqr)
print(f'The most popular visiting interval: {stats['25%']:.2f} - {stats['75%']:.2f}')

Interquartile range (IQR):  9.0
The most popular visiting interval: 13.00 - 22.00
